In [1]:
# Import keras and other Utils
import pltutils as pltu
import matplotlib.pyplot as plt
import numpy as np
import math
import cv2
import tensorflow as tf
import os


from keras.datasets import mnist
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D
from keras.layers import Reshape, MaxPooling2D, Flatten
from keras.layers import Input, InputLayer, Lambda, Dropout
from keras.layers import BatchNormalization,Softmax
from keras.optimizers import RMSprop
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator 
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [3]:
# Create Model
model = Sequential()
model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',input_shape=(224,224,3),activation='relu'))
model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='valid'))
model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
model.add(Conv2D(128,(3,3),padding='same',activation='relu'))

model.add(MaxPooling2D((2,2)))
model.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model.add(Conv2D(256,(3,3),padding='same',activation='relu'))

model.add(MaxPooling2D((2,2)))
model.add(Conv2D(512,(3,3),padding='same',activation='relu'))
model.add(Conv2D(512,(3,3),padding='same',activation='relu'))
model.add(Conv2D(512,(3,3),padding='same',activation='relu'))

model.add(MaxPooling2D((2,2)))
model.add(Conv2D(512,(3,3),padding='same',activation='relu'))
model.add(Conv2D(512,(3,3),padding='same',activation='relu'))
model.add(Conv2D(512,(3,3),padding='same',activation='relu'))

model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dense(4096,activation='relu'))
model.add(Dense(100,activation='softmax'))
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 256)      

In [4]:
import keras
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer = keras.optimizers.SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

In [8]:

batch_size = 32
 

#实例化对象datagen
datagen=ImageDataGenerator()   
 
#读训练集图片
train_generator = datagen.flow_from_directory(
    'e:/tensorflow/data/ImageNet/train100',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=batch_size)

validation_generator = datagen.flow_from_directory(
    'e:/tensorflow/data/ImageNet/val100',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=batch_size)
 

Found 129395 images belonging to 100 classes.
Found 5000 images belonging to 100 classes.


In [9]:
import os
checkpoint_path = "checkpoints/vgg16-train100/checkpoint-weights_only-epoch{epoch:04d}-accuracy{accuracy:.4f}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
print(checkpoint_dir)
cp_callback=ModelCheckpoint(checkpoint_path, verbose=1, save_weights_only=True,period=1)

checkpoints/vgg16-train100


In [7]:
model.save('checkpoints/vgg16-train100/model.h5')

In [12]:
model.fit_generator(generator=train_generator,steps_per_epoch=129395/batch_size,epochs=50
                   ,callbacks=[cp_callback],
                   validation_data=validation_generator,validation_steps=batch_size)


Epoch 1/50
4044/4043 [==============================] - 1836s 454ms/step - loss: 3.7893 - accuracy: 0.1231 - val_loss: 3.3636 - val_accuracy: 0.2100

Epoch 00001: saving model to checkpoints/vgg16-train100/checkpoint-weights_only-epoch0001-accuracy0.1231.ckpt
Epoch 2/50
4044/4043 [==============================] - 1839s 455ms/step - loss: 2.7777 - accuracy: 0.3015 - val_loss: 2.8730 - val_accuracy: 0.3213

Epoch 00002: saving model to checkpoints/vgg16-train100/checkpoint-weights_only-epoch0002-accuracy0.3015.ckpt
Epoch 3/50
4044/4043 [==============================] - 1834s 453ms/step - loss: 2.2143 - accuracy: 0.4222 - val_loss: 2.7168 - val_accuracy: 0.3887

Epoch 00003: saving model to checkpoints/vgg16-train100/checkpoint-weights_only-epoch0003-accuracy0.4222.ckpt
Epoch 4/50
4044/4043 [==============================] - 1832s 453ms/step - loss: 1.7690 - accuracy: 0.5217 - val_loss: 2.1044 - val_accuracy: 0.4277

Epoch 00004: saving model to checkpoints/vgg16-train100/checkpoint-wei

KeyboardInterrupt: 

In [29]:
model.load_weights('./checkpoints/vgg16-train10/checkpoint-weights_only-epoch0016-accuracy0.8942.ckpt')
model.evaluate_generator(generator=validation_generator)

[0.9037704467773438, 0.5199999809265137]